In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cvxopt
import itertools
import copy

%matplotlib inline

## Convex optimisation problem definition

The general convex optimization problem is:
\begin{equation}
\begin{aligned}
\text{minimize } & \frac{1}{2} \mathbf{x}^T \mathbf{P} \mathbf{x} + \mathbf{q}^T\mathbf{x} \\
\text{subject to } & \mathbf{G}\mathbf{x} \leq \mathbf{h} \\
& \mathbf{A}\mathbf{x} = \mathbf{b}.
\end{aligned}
\end{equation}

In the general (soft or hard margin) SVDD formulation, the equations given in Tax, 2004 are as follows:
\begin{equation}
\begin{aligned}
\max_\mathbf{a} \text{ } & \mathcal{L}(\mathbf{a}) = \sum_{n=1}^{N}a_n k(\mathbf{x}_n, \mathbf{x}_n) -\sum_{n=1}^{N}\sum_{m=1}^N a_n a_m k(\mathbf{x}_n, \mathbf{x}_m), \\
&\text{such that } 0 \leq a_n \leq C \\
& \text{and } \sum_{n=1}^N a_i = 1  \\
\end{aligned}
\end{equation}

There is also a formulation for SVDD that includes negative samples. The derivation is slightly more involved, but at the end of the derivation it is revealed that if one defines a variable $a^{'}_i$ as $a^{'}_i = y_i a_i$, we can simply reformulate the problem as 
\begin{equation}
\begin{aligned}
\max_\mathbf{a}^{'} \text{ } & \mathcal{L}(\mathbf{a}^{'}) = \sum_{n=1}^{N}a^{'}_n k(\mathbf{x}_n, \mathbf{x}_n) -\sum_{n=1}^{N}\sum_{m=1}^N a^{'}_n a^{'}_m k(\mathbf{x}_n, \mathbf{x}_m), \\
&\text{such that } 0 \leq a^{'}_n \leq C \\
& \text{and } \sum_{n=1}^N a^{'}_i = 1  \\
\end{aligned}
\end{equation}

Note that the variable $y_i \in \{-1, 1\}$, and $y_i=-1$ is given to samples that the radius should not encompass. Finally, we can calculate the centroid $\alpha$ using
\begin{equation}
\boldsymbol\alpha = \sum a^{'}_i \mathbf{x_i}.
\end{equation}

If we test new data, we can do so using the radius formula
\begin{equation}
R^2 = k(\mathbf{x}_k, \mathbf{x}_k) - 2 \sum_i a^{'}_i k(\mathbf{x}_i, \mathbf{x}_k) + \sum_{i}\sum_{j} k(\mathbf{x}_i, \mathbf{x}_j)
\end{equation}

where $\mathbf{x}_k$ is any support vector. This radius is used to test new data using:
\begin{equation}
\Vert \mathbf{z} - \boldsymbol\alpha \Vert_2^2 = k(\mathbf{z}, \mathbf{z}) - 2 \sum_i a^{'}_i k(\mathbf{z}, \mathbf{x}_k) + \sum_{i}\sum_{j} k(\mathbf{x}_i, \mathbf{x}_j) \leq R^2
\end{equation}

we can subtract $R^2 - \Vert \mathbf{z} - \boldsymbol\alpha \Vert_2^2$ to produce a positive value if a test sample is in the decision boundary, or a negative value otherwise.

We can immediately see that we can rewrite this to be suitable to the convex optimisation problem: let us reformulate the convex problem with $\mathbf{x} = \mathbf{a}$ to give the following matrices (let $\otimes$ be the outer product)

\begin{equation}
\mathbf{P} = 2 * \mathbf{K}(X),
\end{equation}
where $\mathbf{K}$ is the gram matrix:
\begin{equation}
K_{ij} = k(\mathbf{x}_i, \mathbf{x}_j),
\end{equation}

Furthermore, $\mathbf{q}$ is simply:
\begin{equation}
\mathbf{q} = -\text{diag}(\mathbf{K}).
\end{equation}

If we solve to hard margin problem ($C = \infty$, $a_n \geq 0$), then $\mathbf{G}$ is simply the negative identify matrix.
\begin{equation}
\mathbf{G}_{hard} = - \mathbf{I}
\end{equation}
and $\mathbf{h}$ is simply a zero vector
\begin{equation}
\mathbf{h} = \mathbf{0}.
\end{equation}

However, if we solve to soft margin problem ($0\leq a_n \leq C$), $\mathbf{G}$ and $\mathbf{h}$ become
\begin{equation}
\mathbf{G}_{soft} = [\mathbf{G_1}, \mathbf{G_2}]^T
\end{equation}
\begin{equation}
\mathbf{G_1} = -\mathbf{I}
\end{equation}
\begin{equation}
\mathbf{G_2} = \mathbf{I}
\end{equation}
\begin{equation}
\mathbf{h} = [\mathbf{h}_1, \mathbf{h}_2]^T
\end{equation}
\begin{equation}
\mathbf{h}_1 = \mathbf{0}
\end{equation}
\begin{equation}
\mathbf{h}_2 = C * \mathbf{1}
\end{equation}

Finally, the final equality constraint is simple, as both $\mathbf{A} = \mathbf{1}$, where $\mathbf{A}$ is the one vector and $\mathbf{b} = 1$.

We can also demonstrate how the convex problem is formulated for $a^{'}_i = y_ia_i$: let us reformulate the convex problem with $\mathbf{x} = \mathbf{a}$ to give the following matrices (let $\otimes$ be the outer product):
\begin{equation}
\mathbf{P} = 2 * \mathbf{t}\otimes \mathbf{t} * \mathbf{K}(X),
\end{equation}
where $\mathbf{K}$ is the gram matrix:
\begin{equation}
K_{ij} = k(\mathbf{x}_i, \mathbf{x}_j),
\end{equation}

Furthermore, $\mathbf{q}$ is simply:
\begin{equation}
\mathbf{q} = -\mathbf{t} * \text{diag}(\mathbf{K}).T.
\end{equation}

If we solve to hard margin problem ($C = \infty$, $a_n \geq 0$), then $\mathbf{G}$ is simply the negative diagonal matrix with $\mathbf{t}$ on the diagonal.
\begin{equation}
\mathbf{G}_{hard} = - \text{diag}(\mathbf{t})
\end{equation}
and $\mathbf{h}$ is simply a zero vector
\begin{equation}
\mathbf{h} = \mathbf{0}.
\end{equation}

However, if we solve to soft margin problem ($0\leq a_n \leq C$), $\mathbf{G}$ and $\mathbf{h}$ become
\begin{equation}
\mathbf{G}_{soft} = [\mathbf{G_1}, \mathbf{G_2}]^T
\end{equation}
\begin{equation}
\mathbf{G_1} = -\text{diag}(\mathbf{t})
\end{equation}
\begin{equation}
\mathbf{G_2} = \text{diag}(\mathbf{t})
\end{equation}
\begin{equation}
\mathbf{h} = [\mathbf{h}_1, \mathbf{h}_2]^T
\end{equation}
\begin{equation}
\mathbf{h}_1 = \mathbf{0}
\end{equation}
\begin{equation}
\mathbf{h}_2 = C * \mathbf{1}
\end{equation}

Finally, the final equality constraint is simple, as both $\mathbf{A} = \mathbf{1}$, where $\mathbf{A}$ is the one vector and $\mathbf{b} = 1$.

## SVDD Object

In [ ]:
class SVDD(object):
    def __init__(self, kernel = 'linear', degree = 3, sigma = 1, C = None, alpha_threshold = 1e-5):
        
        self.kernel = kernel
        self.degree = degree #polynomial kernel degree
        self.sigma = sigma #Gaussian kernel 1/variance
        self.C = C #1/N <= C <= 1
        self.alpha_threshold = alpha_threshold
        self.radius_threshold = 1e-4

    def linear_kernel(self, x1, x2):
        return np.dot(x1, x2)
    
    def polynomial_kernel(self, x1, x2):
        return (1 + np.dot(x1, x2)) **self.degree
    
    def gaussian_kernel(self, x1, x2):
        return np.exp(-1 * (np.linalg.norm(x1 - x2)**2) / self.sigma**2 )
    
    def apply_kernel_function(self, x1, x2):
        
        if self.kernel == 'linear':
            return self.linear_kernel(x1, x2)
        
        elif self.kernel == 'polynomial':
            return self.polynomial_kernel(x1, x2)
            
        elif self.kernel == 'gaussian':
            return self.gaussian_kernel(x1, x2)
        
        else:
            print("Illegal kernel function chosen.")
            raise SystemExit
    
    def calculate_gram(self, X):
        n, f = X.shape
        
        K = np.zeros((n, n))
        
        for i in range(n):
            for j in range(n):
                K[i, j] = self.apply_kernel_function(X[i, :], X[j, :])
        
        return K
    
    def calculate_max_dist(self, X):
        n, f = X.shape
        
        dist_mat = np.zeros((n, n))
        
        for i in range(n):
            dist_mat[i, :] = np.sqrt(np.sum((X[i, :] - X)**2, axis = 1))
            #for j in range(n):
                #dist_mat[i, j] = np.linalg.norm(X[i, :] - X[j, :])**2
        
        return np.max(np.sqrt(dist_mat))
    
    def fit(self, X, y = None):
        
        #labels can be None, but 
        n = X.shape[0]
        
        if y is None:
            y = np.ones(n)
            
        else:
            assert len(y) == n, "The labels are not the same shape as the input data X."
        
        #Determine gram matrix
        Gram = self.calculate_gram(X)
        
        #Setup necessary matrices to be used for lagrangian multipliers
        #P and q
        
        P = cvxopt.matrix(2 * np.outer(y, y) *  Gram)
        q = cvxopt.matrix(-1 * y.reshape(-1, 1) * np.diag(Gram).reshape(-1, 1))
        
        #G and h
        if self.C is None:
            print("\nSolving the hard margin problem!")
            G = cvxopt.matrix(-1 * np.diag(y))
            h = cvxopt.matrix(np.zeros((n, 1)))
            
        else:
            print("\nSolving the soft margin problem!")
            G_1 = -1 * np.diag(y)
            h_1 = np.zeros((n, 1))
            
            G_2 = np.diag(y)
            h_2 = self.C * np.ones((n, 1))
            
            G = cvxopt.matrix(np.vstack((G_1, G_2)))
            h = cvxopt.matrix(np.vstack((h_1, h_2)))
        
        #A and b
        A = cvxopt.matrix(y.reshape(1, -1))
        b = cvxopt.matrix(np.ones((1, 1)))
        
        #We are now ready to solve the convex optimisation problem!
        self.solution = cvxopt.solvers.qp(P, q, G, h, A, b)  
        
        #Save alphas
        self.alphas = np.array(self.solution['x'])[:, 0]
        
        #Setup index lists!
        support_alpha_indices = []
        
        for cnt, i in enumerate(self.alphas):
            if i >= self.alpha_threshold:
                if self.C is not None:
                     if i != self.C: #SV = 0 < alpha_i < C
                        support_alpha_indices.append(cnt)
                
                else:
                    support_alpha_indices.append(cnt)
                
                        
        
        self.support_alpha_indices = support_alpha_indices
        self.support_alphas = self.alphas[self.support_alpha_indices]
        self.support_vectors = X[self.support_alpha_indices, :]
        self.support_labels = y[self.support_alpha_indices]
        self.sv_weights = self.support_alphas * self.support_labels #alpha_prime for sv's
        
        #Calculate center
        self.center = np.sum(self.sv_weights.reshape(-1, 1) * self.support_vectors, axis = 0)
        
        #Calculate radius
        sv_index = 0 #Use first support vector to compute radius
        actual_index = self.support_alpha_indices[sv_index]
        
        #Compute sv_gram (will simply model evaluation and prediction)
        self.Gram_svs = self.calculate_gram(self.support_vectors)
        
        t1 = Gram[actual_index, actual_index]
        t2 = -2 * np.dot(self.sv_weights, self.Gram_svs[sv_index, :])
        t3 = np.dot(self.sv_weights.reshape(-1, 1).T, np.dot(self.Gram_svs, self.sv_weights.reshape(-1, 1)))
        
        self.square_radius = t1 + t2 + t3
    
    @staticmethod
    def create_indices(N, start_fold, size_fold):
        
        idx_train = list(range(0, start_fold, 1)) + list(range(start_fold + size_fold, N, 1))
        idx_test = list(range(start_fold, start_fold + size_fold, 1))
        
        return idx_train, idx_test
    
    def optimise_parameters(self, X, y = None, sigma_low = 1e-4, C_opt = False, N_iter = 50, loocv_flag = True, k_fold = None, plot_flag = False):
        
        N = X.shape[0]
        
        if y is None:
            y = np.ones(N)
            
        else:
            assert len(y) == N, "The labels are not the same shape as the input data X."
        
        #Check that the user hasn't broken anything
        if not loocv_flag: #k-fold loop
            
            assert k_fold is not None, "k_fold variable should be an integer, not {}.".format(k_fold)
            size_fold = N // k_fold
            
        #Calculate the maximum sigma value that can be obtained
        max_sigma = self.calculate_max_dist(X)
        
        #Define the sigma range
        sigma_range = np.linspace(sigma_low, max_sigma, N_iter)
        
        #Set up the grid search points
        if C_opt:#1/N leq C leq 1
            if not loocv_flag:
                C_range = np.linspace(1/(N - size_fold), 1, N_iter)  #Adjust because of fold size
            
            else:
                C_range = np.linspace(1/(N), 1, N_iter)
        else:
            C_range = np.array([self.C])
            
        iter_list = list(itertools.product(C_range, sigma_range))
        
        #Define the loss array
        loss_array = np.zeros(len(iter_list))
        
        #Save the original parameters
        self.orig_params = copy.deepcopy(self.get_params())
        
        for cnt, iter_vals in enumerate(iter_list):
            
            print("\nBeginning optimisation iteration {}...".format(cnt + 1))
            
            #Get parameters
            self.C = iter_vals[0]
            self.sigma = iter_vals[1]
            
            if loocv_flag: #Simple loocv loop
                
                #fit
                self.fit(X, y)
                
                #score
                loss_array[cnt], _, _ = self.score(X, y, error_loocv = True)
            
            else: #k-fold loop
                
                #Get parameters
                self.C = iter_vals[0]
                self.sigma = iter_vals[1]
                
                loss_k_fold = np.zeros(k_fold)
                cnt_k_fold = 0
                start_fold = 0
                
                for k_index in range(k_fold):
                    
                    print("\nBeginning k-fold iteration {}...".format(k_index + 1))
                    
                    train_indices, test_indices = self.create_indices(N, start_fold, size_fold)
                    
                    X_train, X_test, y_train, y_test = X[train_indices], X[test_indices], y[train_indices], y[test_indices]
                    
                    plt.figure()
                    plt.scatter(X_train[:, 0], X_train[:, 1], marker = "x", color = "r")
                    plt.scatter(X_test[:, 0], X_test[:, 1], marker = "d", color = "b")
                    plt.show()
                    
                    #fit
                    self.fit(X_train, y_train)
                    
                    #score
                    loss_k_fold[cnt_k_fold], _, _ = self.score(X_test, y_test, error_loocv = False)
                    
                    cnt_k_fold += 1
                    start_fold += size_fold
                
                #Score
                loss_array[cnt] = np.mean(loss_k_fold)  
        
        self.opt_losses = loss_array
        
        opt_index = np.argmin(self.opt_losses)
        
        opt_dict = {"C":iter_list[opt_index][0], "sigma":iter_list[opt_index][1]}
        
        self.opt_params = opt_dict
        
        #Visualise
        if plot_flag:
            if C_opt:
                x_vis, y_vis = np.meshgrid(C_range, sigma_range)
                
                plt.figure()
                plt.contourf(x_vis, y_vis, self.opt_losses.reshape(N_iter, N_iter))
                plt.xlabel("C")
                plt.ylabel(r"$\sigma$")
                plt.show()
                
            else:
                plt.figure()
                plt.plot(sigma_range, self.opt_losses)
                plt.xlabel(r"$\sigma$")
                plt.ylabel("Objective")
                plt.show()
        
        return opt_dict #C, sigma
    
    def evaluate_model(self, x):
        #X = (N,) array
        
        pred_val = 0
        
        t1_pred = self.apply_kernel_function(x, x)
        
        t2_pred = 0
        
        for cnt, i in enumerate(self.sv_weights):
            t2_pred += i * self.apply_kernel_function(x, self.support_vectors[cnt, :])
        
        t2_pred *= -2
        
        t3_pred = np.dot(self.sv_weights.reshape(-1, 1).T, np.dot(self.Gram_svs, self.sv_weights.reshape(-1, 1)))
        
        return t1_pred + t2_pred + t3_pred
    
    def predict(self, X, y = None):

        if len(X.shape) > 1:

            n = X.shape[0]
            prediction = np.zeros(n)

            for i in range(n):
                prediction[i] = self.square_radius - self.evaluate_model(X[i, :])

        else:
            prediction = self.square_radius - self.evaluate_model(X)

        return prediction

    def score(self, X, y = None, error_loocv = True):
        N = X.shape[0]
        
        pred = self.predict(X, y)
        
        if y is not None:
            pred *= y #Adjust for correctly labelled data
        
        #Calculate the error:
        error1 = np.mean(pred < -1 * self.radius_threshold) #misclassification error
        error2 = len(self.support_alpha_indices) / N #Loocv error
        
        if error_loocv: 
            cost = np.sqrt(error2**2 + np.abs(1 - np.sqrt(self.square_radius))**2)
        
        else:
            cost = np.sqrt(error1**2 + np.abs(1 - np.sqrt(self.square_radius))**2)
        
        return cost, error1, error2
    
    def get_params(self):
        param_dict = {"C":self.C, "sigma":self.sigma}
        
        return param_dict
    
    def set_params(self, param_dict):
        self.C = param_dict["C"]
        self.sigma = param_dict["sigma"]
        
        return self
        
        


## Generate dataset 1

In [ ]:
N_sample = 1000
R = 4
x_data = np.random.random(N_sample) * 8 - 4
y_data = np.random.random(N_sample) * 8 - 4

X_data = np.vstack((x_data, y_data)).T

R_data = np.sum(X_data**2, axis = 1)

labels = 2 * (R_data < R**2) - 1

x_plot = np.linspace(-4, 4, 400)

plt.figure()
plt.scatter(X_data[:, 0], X_data[:, 1], c = labels)
plt.plot(x_plot, np.sqrt(16 - x_plot**2), "r--")
plt.plot(x_plot, -np.sqrt(16 - x_plot**2), "r--")
plt.show()

#Get data only in sphere
X = X_data[np.nonzero(R_data < R**2)[0], :]

plt.figure()
plt.title("Data")
plt.scatter(X[:, 0], X[:, 1])
plt.plot(x_plot, np.sqrt(16 - x_plot**2), "r--")
plt.plot(x_plot, -np.sqrt(16 - x_plot**2), "r--")
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$")
plt.show()


If we test new data, we can do so using the radius formula
\begin{equation}
R^2 = k(\mathbf{x}_k, \mathbf{x}_k) - 2 \sum_i a^{'}_i k(\mathbf{x}_i, \mathbf{x}_k) + \sum_{i}\sum_{j} k(\mathbf{x}_i, \mathbf{x}_j)
\end{equation}

where $\mathbf{x}_k$ is any support vector. This radius is used to test new data using:
\begin{equation}
\Vert \mathbf{z} - \boldsymbol\alpha \Vert_2^2 = k(\mathbf{z}, \mathbf{z}) - 2 \sum_i a^{'}_i k(\mathbf{z}, \mathbf{x}_k) + \sum_{i}\sum_{j} k(\mathbf{x}_i, \mathbf{x}_j) \leq R^2
\end{equation}

we can subtract $R^2 - \Vert \mathbf{z} - \boldsymbol\alpha \Vert_2^2$ to produce a positive value if a test sample is in the decision boundary, or a negative value otherwise.

## Train model on dataset 1

In [ ]:
svdd_test = SVDD('linear', C = 1)
svdd_test.fit(X)

plt.figure()
plt.title("Alphas - dataset 1")
plt.scatter(np.arange(len(svdd_test.alphas)), svdd_test.alphas)
plt.show()

print(svdd_test.center, svdd_test.square_radius)

In [ ]:
svdd_test.score(X)

### Visualise results

In [ ]:
N_grid = 100
X_m, Y_m = np.meshgrid(np.linspace(np.min(X[:, 0]) - 2, np.max(X[:, 0]) + 2, N_grid), np.linspace(np.min(X[:, 1]) - 2, np.max(X[:, 1]) + 2, N_grid))

X_grid = np.hstack((X_m.reshape(-1, 1), Y_m.reshape(-1, 1)))

Z = svdd_test.predict(X_grid)

plt.figure()
plt.contour(X_m, Y_m, Z.reshape(N_grid, N_grid))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_test.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title(r"Contours - y($\mathbf{x}$)")

plt.show()

plt.figure()
plt.contourf(X_m, Y_m, np.sign(Z.reshape(N_grid, N_grid)))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_test.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title("On/Off visualisation")
plt.show()

## Generate dataset 2 - including positive and negative labels

In [ ]:
X1 = np.random.randn(100, 2) + np.array([4, 4])
X2 = np.random.randn(100, 2) + np.array([-4, -4])
X3 = np.random.randn(100, 2) + np.array([-3, 3])
X4 = np.random.randn(100, 2) + np.array([3, -3])

l1 = np.ones(100)
l2 = np.ones(100)
X = np.vstack((X1, X2, X3, X4))
labels = np.hstack((l1, l2, -l1, -l2))


plt.figure()
plt.title("Data")
plt.scatter(X[:, 0], X[:, 1], c = labels)
plt.xlabel(r"$x_1$", fontsize = 14)
plt.ylabel(r"$x_2$", fontsize = 14)
plt.show()


### Train model on dataset 2

In [ ]:
svdd_test = SVDD('gaussian', C = 0.8, sigma = 1)
#opt_params = svdd_test.optimise_parameters(X, y = labels, sigma_low = 1e-4, C_opt = True, N_iter = 10, plot_flag = True)

#print(opt_params)
#set params
#svdd_test.set_params(opt_params)

#Fit model
svdd_test.fit(X, labels)

plt.figure()
plt.scatter(np.arange(len(svdd_test.alphas)), svdd_test.alphas * labels)
plt.show()


### Visualise results

In [ ]:
N_grid = 100
X_m, Y_m = np.meshgrid(np.linspace(np.min(X[:, 0]) - 2, np.max(X[:, 0]) + 2, N_grid), np.linspace(np.min(X[:, 1]) - 2, np.max(X[:, 1]) + 2, N_grid))

X_grid = np.hstack((X_m.reshape(-1, 1), Y_m.reshape(-1, 1)))

Z = svdd_test.predict(X_grid)

plt.figure()
plt.contour(X_m, Y_m, Z.reshape(N_grid, N_grid))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_test.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title(r"Contours - y($\mathbf{x}$)")

plt.show()

plt.figure()
plt.contourf(X_m, Y_m, np.sign(Z.reshape(N_grid, N_grid)))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_test.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title("On/Off visualisation")
plt.show()

In [ ]:
svdd_test.score(X, labels), svdd_test.square_radius

## Generate dataset 3 - For optimising hyper-parameters without labels

In [ ]:
N_data = 400
X1 = np.random.randn(N_data, 2) + np.array([4, 4])
X2 = np.random.randn(N_data, 2) + np.array([-4, -4])
percentage = 0.8

l1 = np.ones(N_data)
l2 = np.ones(N_data)
X = np.vstack((X1, X2))
labels = np.hstack((l1, l2))

range_all = np.arange(X.shape[0])
np.random.shuffle(range_all)

X = X[range_all]

X_train = X[:int(percentage * X.shape[0]), :] 
labels_train = labels[:int(percentage * X.shape[0])]

X_test = X[int(percentage * X.shape[0]):, :] 
labels_test = labels[int(percentage * X.shape[0]):]

#X_test = np.vstack((X1, X2, X3, X4))
#labels_test = np.hstack((l1, l2, -1 *  l1, -1 *  l2))

plt.figure()
plt.title("Data")
plt.scatter(X_train[:, 0], X_train[:, 1], c = labels_train)
plt.xlabel(r"$x_1$", fontsize = 14)
plt.ylabel(r"$x_2$", fontsize = 14)
plt.show()

plt.figure()
plt.title("Data - testing")
plt.scatter(X_test[:, 0], X_test[:, 1], c = labels_test)
plt.xlabel(r"$x_1$", fontsize = 14)
plt.ylabel(r"$x_2$", fontsize = 14)
plt.show()

## Varying sigma and plotting the objective function from Theissler and Dear, 2013

In [ ]:
N_CV = 50
cost1_sigma = np.zeros(N_CV)
cost2_sigma = np.zeros(N_CV)
error1_sigma = np.zeros(N_CV)
error2_sigma = np.zeros(N_CV)
radius_sigma = np.zeros(N_CV)
set_C = 1

#sigma variation
sigma_array = np.linspace(1e-4, 25, N_CV)

for cnt, sigma_cross in enumerate(sigma_array):
    
    svdd_test = SVDD('gaussian', C = set_C, sigma = sigma_cross)
    svdd_test.fit(X_train)

    cost1_sigma[cnt], error1_sigma[cnt], error2_sigma[cnt] = svdd_test.score(X_test, error_loocv = True)
    cost2_sigma[cnt], _, _ = svdd_test.score(X_test, error_loocv = False)
    
    radius_sigma[cnt] = np.sqrt(svdd_test.square_radius[0, 0])

### Visualise response for optimal sigma model

In [ ]:
plt.figure()
plt.title("Loss - Sigma")
plt.semilogx(sigma_array, cost1_sigma, label = "Theissler and Dear (2013) loss (loocv for Ew)")
plt.semilogx(sigma_array, cost2_sigma, label = "Theissler and Dear (2013) loss (data for Ew)")
plt.legend()
plt.grid()
plt.show()

plt.figure()
plt.title("Errors - Sigma")
plt.semilogx(sigma_array, error1_sigma, label = "Error - trained")
plt.semilogx(sigma_array, error2_sigma, label = "SV error (approximation)")
plt.legend()
plt.grid()
plt.show()

plt.figure()
plt.title("Radius - Sigma")
plt.semilogx(sigma_array, radius_sigma, label = "Radius")
plt.legend()
plt.grid()
plt.show()

print("What we can see here is that you NEED cross validation if you try to estimate the best fit from all the data")
print("Clearly, you overfit heavily for the case where you work out the error based on the data, because for small sigmas all the points become SVs")

svdd_sigma = SVDD('gaussian', C = 10, sigma = sigma_array[np.argmin(cost1_sigma)])
svdd_sigma.fit(X)

N_grid = 100
X_m, Y_m = np.meshgrid(np.linspace(np.min(X[:, 0]) - 2, np.max(X[:, 0]) + 2, N_grid), 
                       np.linspace(np.min(X[:, 1]) - 2, np.max(X[:, 1]) + 2, N_grid))

X_grid = np.hstack((X_m.reshape(-1, 1), Y_m.reshape(-1, 1)))

Z = svdd_sigma.predict(X_grid)

plt.figure()
plt.contour(X_m, Y_m, Z.reshape(N_grid, N_grid))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_sigma.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title(r"Contours - y($\mathbf{x}$)")

plt.show()

plt.figure()
plt.contourf(X_m, Y_m, np.sign(Z.reshape(N_grid, N_grid)))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_sigma.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title("On/Off visualisation")
plt.show()

## Varying the C parameter and plotting the objective function from Theissler and Dear, 2013

In [ ]:
N_CV = 50
cost1_C = np.zeros(N_CV)
cost2_C = np.zeros(N_CV)
error1_C = np.zeros(N_CV)
error2_C = np.zeros(N_CV)
set_sigma = sigma_array[np.argmin(cost1_sigma)] #Use previous solution
radius_C = np.zeros(N_CV)

#sigma variation
C_array = np.linspace(1/X_train.shape[0], 1, N_CV)

for cnt, C_cross in enumerate(C_array):
    svdd_test = SVDD('gaussian', C = C_cross, sigma = set_sigma)
    svdd_test.fit(X)
    
    cost1_C[cnt], error1_C[cnt], error2_C[cnt] = svdd_test.score(X_test, error_loocv = True)
    cost2_C[cnt], _, _ = svdd_test.score(X_test, error_loocv = False)
    
    radius_C[cnt] = np.sqrt(svdd_test.square_radius[0, 0])

### Visualise response for optimal C model

In [ ]:
plt.figure()
plt.title("Loss - C")
plt.plot(C_array, cost1_C, label = "Theissler and Dear (2013) loss (loocv for Ew)")
plt.plot(C_array, cost2_C, label = "Theissler and Dear (2013) loss (data for Ew)")
plt.legend()
plt.grid()
plt.show()

plt.figure()
plt.title("Errors - C")
plt.plot(C_array, error1_C, label = "SV error (approximation)")
plt.plot(C_array, error2_C, label = "Error - trained")
plt.legend()
plt.grid()
plt.show()

plt.figure()
plt.title("Radius - C")
plt.plot(C_array, radius_C, label = "Radius")
plt.legend()
plt.grid()
plt.show()

print("Same analysis here.")

svdd_C = SVDD('gaussian', C = C_array[np.argmin(cost1_C)], sigma = sigma_array[np.argmin(cost1_sigma)])
svdd_C.fit(X)

N_grid = 100
X_m, Y_m = np.meshgrid(np.linspace(np.min(X[:, 0]) - 2, np.max(X[:, 0]) + 2, N_grid), 
                       np.linspace(np.min(X[:, 1]) - 2, np.max(X[:, 1]) + 2, N_grid))

X_grid = np.hstack((X_m.reshape(-1, 1), Y_m.reshape(-1, 1)))

Z = svdd_C.predict(X_grid)

plt.figure()
plt.contour(X_m, Y_m, Z.reshape(N_grid, N_grid))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_C.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title(r"Contours - y($\mathbf{x}$)")

plt.show()

plt.figure()
plt.contourf(X_m, Y_m, np.sign(Z.reshape(N_grid, N_grid)))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_C.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title("On/Off visualisation")
plt.show()

## Summary of varying sigma and C parameters

What I found after this investigation is that the LOOCV approach to calculating $E_w$ ($E_w = \frac{\#SV}{N}$) is a pretty good way of calculating the loss from Theissler and Dear (2013). I summarise my main conclusions here:

- There does not appear to be a big improvement between using the LOOCV approximation to $E_w$ and the exact approach.
- What was of CRUCIAL importance for the Theissler approach is that one uses a test set that is not like the test set to optimise the model. The reason for this is that at low $\sigma$, all SVs are used to define the decision boundary. Hence, if you use the same data for training and testing, $E_w$ will be low but the model is poor.

## Optimising the hyper-parameters - Using the LOOCV error solution (thus not needing to do cross-validation)

If loocv_flag = True, then there is no k-fold cross-validation. The SVDD object is designed to perform k-fold cross validation internally, if required.

In [ ]:
svdd_loocv = SVDD('gaussian', C = 0.1, sigma = 1)
opt_params = svdd_loocv.optimise_parameters(X, 
                                           y = None, 
                                           sigma_low = 1e-4,
                                           C_opt = True, 
                                           N_iter = 10, 
                                           loocv_flag = True,
                                           k_fold = None,
                                           plot_flag = True)

print(opt_params)

#set params
svdd_loocv.set_params(opt_params)

#Fit model to all the data
svdd_loocv.fit(X)

### Visualise results

In [ ]:
N_grid = 100
X_m, Y_m = np.meshgrid(np.linspace(np.min(X[:, 0]) - 2, np.max(X[:, 0]) + 2, N_grid), 
                       np.linspace(np.min(X[:, 1]) - 2, np.max(X[:, 1]) + 2, N_grid))

X_grid = np.hstack((X_m.reshape(-1, 1), Y_m.reshape(-1, 1)))

Z = svdd_loocv.predict(X_grid)

plt.figure()
plt.contour(X_m, Y_m, Z.reshape(N_grid, N_grid))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_loocv.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title(r"Contours - y($\mathbf{x}$)")

plt.show()

plt.figure()
plt.contourf(X_m, Y_m, np.sign(Z.reshape(N_grid, N_grid)))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_loocv.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title("On/Off visualisation")
plt.show()

## Optimising the hyper-parameters - Using the k-fold solution (thus performing cross-validation)

In [ ]:
svdd_kfold = SVDD('gaussian', C = 0.1, sigma = 1)

opt_params = svdd_kfold.optimise_parameters(X, 
                                           y = None, 
                                           sigma_low = 1e-4,
                                           C_opt = True, 
                                           N_iter = 10, 
                                           loocv_flag = False,
                                           k_fold = 5,
                                           plot_flag = True)

print(opt_params)

#set params
svdd_kfold.set_params(opt_params)

#Fit model to all the data
svdd_kfold.fit(X)

### Visualise results

In [ ]:
N_grid = 100
X_m, Y_m = np.meshgrid(np.linspace(np.min(X[:, 0]) - 2, np.max(X[:, 0]) + 2, N_grid), 
                       np.linspace(np.min(X[:, 1]) - 2, np.max(X[:, 1]) + 2, N_grid))

X_grid = np.hstack((X_m.reshape(-1, 1), Y_m.reshape(-1, 1)))

Z = svdd_kfold.predict(X_grid)

plt.figure()
plt.contour(X_m, Y_m, Z.reshape(N_grid, N_grid))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_kfold.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title(r"Contours - y($\mathbf{x}$)")

plt.show()

plt.figure()
plt.contourf(X_m, Y_m, np.sign(Z.reshape(N_grid, N_grid)))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_kfold.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title("On/Off visualisation")
plt.show()

In [ ]:
svdd_loocv.opt_losses, "\n", svdd_kfold.opt_losses

## Comments at the end

In [ ]:
svdd_loocv.opt_params, "\n", svdd_kfold.opt_params

Interestingly, for this problem, the optimal dictionaries for the k-fold and the LOOCV optimal solution are similar, and the optimal values are very close (only the C value is different, but this is expected as 1/N is different for k-fold (subtract the test set length). Finally, lets run the same code but with labels, just to be sure everything works!

## Dataset 4 - optimisation with labels

### Generate dataset

In [ ]:
X1 = np.random.randn(100, 2) + np.array([4, 4])
X2 = np.random.randn(100, 2) + np.array([-4, -4])
X3 = np.random.randn(100, 2) + np.array([-3, 3])
X4 = np.random.randn(100, 2) + np.array([3, -3])

l1 = np.ones(100)
l2 = np.ones(100)
X = np.vstack((X1, X2, X3, X4))
labels = np.hstack((l1, l2, -l1, -l2))

reorder = np.arange(X.shape[0])
np.random.shuffle(reorder)

X = X[reorder, :]
labels = labels[reorder]


plt.figure()
plt.title("Data")
plt.scatter(X[:, 0], X[:, 1], c = labels)
plt.xlabel(r"$x_1$", fontsize = 14)
plt.ylabel(r"$x_2$", fontsize = 14)
plt.show()

### LOOCV

In [ ]:
svdd_loocv = SVDD('gaussian', C = 0.1, sigma = 1)
opt_params = svdd_loocv.optimise_parameters(X, 
                                           y = labels, 
                                           sigma_low = 1e-4,
                                           C_opt = True, 
                                           N_iter = 10, 
                                           loocv_flag = True,
                                           k_fold = None,
                                           plot_flag = True)

print(opt_params)

#set params
svdd_loocv.set_params(opt_params)

#Fit model to all the data
svdd_loocv.fit(X, labels)

In [ ]:
N_grid = 100
X_m, Y_m = np.meshgrid(np.linspace(np.min(X[:, 0]) - 2, np.max(X[:, 0]) + 2, N_grid), 
                       np.linspace(np.min(X[:, 1]) - 2, np.max(X[:, 1]) + 2, N_grid))

X_grid = np.hstack((X_m.reshape(-1, 1), Y_m.reshape(-1, 1)))

Z = svdd_loocv.predict(X_grid)

plt.figure()
plt.contour(X_m, Y_m, Z.reshape(N_grid, N_grid))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_loocv.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title(r"Contours - y($\mathbf{x}$)")

plt.show()

plt.figure()
plt.contourf(X_m, Y_m, np.sign(Z.reshape(N_grid, N_grid)))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_loocv.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title("On/Off visualisation")
plt.show()

### K-fold

In [ ]:
svdd_kfold = SVDD('gaussian', C = 0.1, sigma = 1)

opt_params = svdd_kfold.optimise_parameters(X, 
                                           y = labels, 
                                           sigma_low = 1e-4,
                                           C_opt = True, 
                                           N_iter = 10, 
                                           loocv_flag = False,
                                           k_fold = 5,
                                           plot_flag = True)

print(opt_params)

#set params
svdd_kfold.set_params(opt_params)

#Fit model to all the data
svdd_kfold.fit(X, labels)

In [ ]:
N_grid = 100
X_m, Y_m = np.meshgrid(np.linspace(np.min(X[:, 0]) - 2, np.max(X[:, 0]) + 2, N_grid), 
                       np.linspace(np.min(X[:, 1]) - 2, np.max(X[:, 1]) + 2, N_grid))

X_grid = np.hstack((X_m.reshape(-1, 1), Y_m.reshape(-1, 1)))

Z = svdd_kfold.predict(X_grid)

plt.figure()
plt.contour(X_m, Y_m, Z.reshape(N_grid, N_grid))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_kfold.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title(r"Contours - y($\mathbf{x}$)")

plt.show()

plt.figure()
plt.contourf(X_m, Y_m, np.sign(Z.reshape(N_grid, N_grid)))
plt.colorbar()
plt.scatter(X[:, 0], X[:, 1], c = svdd_kfold.predict(X), cmap = plt.cm.jet)
#plt.xlabel()
#plt.ylabel()
plt.title("On/Off visualisation")
plt.show()